### Запуск QtLab

In [12]:
%matplotlib qt5

In [22]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
import pandas as pd
interactive(True)

In [2]:
#import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)
import qsweepy.save_pkl
imp.reload(qsweepy.save_pkl)
from qsweepy.save_pkl import *

In [23]:
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

In [5]:
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [25]:
switch_sample = {1:1, 5:2, 6:3, 4:0}

# Прицеливание

In [26]:
sample_name = 'Res BMTSU_MISiS Nb 17.10.18 sample #0'.format(switch_sample[rf_switch.get_ch2_switch()])
resonator_type='reflection_port'
min_freq = 6.5e9
max_freq = 8e9
data_root = get_location()

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [25]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(7501)
pna.set_bandwidth(0.5e3)
pna.set_power(-35)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


True

In [67]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.gradient(phase)/(freqs[1]-freqs[0])
#delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
delay_freqs=freqs
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [27]:
sweep.sweep(pna, ([0], lambda x: 0, ''), filename="S21 overview "+sample_name)

HTML(value='')

Started at:  Oct 17 2018 07:50:06


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2185: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2186: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2188: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)


Exiting sweep main thread #{} via return 9268

Elapsed time: 0 h 0 m 18.08 s


{'S-parameter': (('Frequency',),
  (array([6.5000e+09, 6.5002e+09, 6.5004e+09, ..., 7.9996e+09, 7.9998e+09,
          8.0000e+09]),),
  array([-0.10634194+1.05460024j, -0.03410663+1.05863488j,
          0.03184176+1.05920649j, ..., -0.42863694+0.68543911j,
         -0.38111106+0.71872729j, -0.3334966 +0.73786551j]),
  {'log': 20},
  ('Hz',))}

In [73]:
threshold = -2.5e-7

In [74]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
from scipy.signal import argrelextrema
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = delay
data_diff[data_diff<threshold] = 0
#data_diff[data_diff>-threshold] = 0
plt.figure('delay')
plt.plot(freqs, data_diff)
data_diff=np.abs(data_diff)
#peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
neighborhood_size = int(np.floor(2e6/(freqs[1]-freqs[0])))
peaks = argrelextrema(data_diff, np.greater, order=neighborhood_size)[0]
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[peaks]
for freq_id, freq in enumerate(peak_freqs):
    print (freq, ',')

found 5 peaks
7009200000.0 ,
7150000000.0 ,
7286000000.0 ,
7422600000.0 ,
7602000000.0 ,


In [75]:
#rescan peaks individually and fit via circlefit
from resonator_tools import circuit

typical_span=3e6
bw=10e3
nop = typical_span/bw
#pna.set_bandwidth(bw)
pna.set_nop(nop)
pna.set_span(typical_span)
pna.pre_sweep()
high_power_fits = []
for freq_id, freq in enumerate(peak_freqs):
    pna.set_centerfreq(freq)
    measurement = pna.measure()['S-parameter']
    points = pna.get_points()['S-parameter'][0][1]
    delay = np.gradient(measurement)
    fitter = circuit.notch_port(f_data = points, z_data_raw=measurement)
    fitter.autofit()
    high_power_fits.append(fitter.fitresults)
    plt.figure(freq_id)
    fitter.plotall()
pna.post_sweep()
high_power_fits = pd.DataFrame(high_power_fits)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


In [76]:
high_power_fits = pd.DataFrame(high_power_fits)
resonator_freqs = high_power_fits['fr']
resonator_freq_spans = (high_power_fits['fr']/high_power_fits['Ql']*5).tolist()

In [80]:
resonator_freqs[8] = 7.0525e9
resonator_freq_spans[8] = 5e6
resonator_freq_spans[9] = 5e6
resonator_freq_spans[22] = 8.583e6
resonator_freq_spans[22] = 10e6

In [78]:
resonator_freqs, resonator_freq_spans

(0    7.009115e+09
 1    7.149971e+09
 2    7.286019e+09
 3    7.422963e+09
 4    7.602057e+09
 Name: fr, dtype: float64,
 [444918.0850179877,
  1174663.6501668873,
  2063401.547361251,
  4341937.811773582,
  10390855.403199222])

# Мерить зонаторы

In [16]:
resonator_freqs = [ 7.009115e+09, 7.149971e+09, 7.286019e+09, 7.422963e+09, 7.602057e+09]


In [83]:
#resonator_freqs = [ 
#6.6532e9
#6.8037e9,    
#]
#powers = np.linspace(-10, -45, 36) # длинный скан
#powers = np.linspace(-20, -40, 41) # короткий скан

In [89]:
#powers = np.linspace(0, -30, 31) # короткий скан
powers = np.linspace(-15, -75, 61) # длинный скан

In [90]:
resonator_ids = [i for i,f in enumerate(resonator_freqs)]
#resonator_ids = np.asarray([22, 23])-1
resonator_ids

[0, 1, 2, 3, 4]

In [91]:
len(resonator_freqs)

5

In [92]:
resonator_freq_span = 1e6
#resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(30)
    
    
    pna.set_bandwidth(1)
    pna._visainstrument.write("SENS:AVER:MODE POIN")

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    default_measurement_save_path(save_path, mkdir=True)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


HTML(value='')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Started at:  Oct 17 2018 07:59:49
Exiting sweep main thread #{} via return 4308

Elapsed time: 1 h 11 m 28.25 s


C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10


...

C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10


HTML(value='')

Started at:  Oct 17 2018 12:46:01


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


Exiting sweep main thread #{} via return 4376

Elapsed time: 1 h 11 m 27.57 s


# load last measurements and refit

In [2]:
%matplotlib qt5
import resonator_tools.circuit
from qsweepy import *
import pickle
import matplotlib.pyplot as plt

In [27]:
save_path=r'C:\Data\2018-10-17'

In [28]:
import glob
measurement_pattern = r'C:\Data\2018-10-17\*-Res BMTSU_MISiS Nb 17.10.18 sample #0 res {}\*.pkl'
for resonator_id in range(5):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        print (last_measurement)
        with open(last_measurement, 'rb') as f:
            
            measurement = pickle.load(f)[1]['S-parameter']
            powers = measurement[1][0]
            fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port', delay=49e-9)
            fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
            #fitter = resonator_tools.circuit.reflection_port(f_data =measurement[1][1], z_data_raw=measurement[2][-1,:])
            #fitter.GUIfit()
            #plt.close()
            #plt.close()

C:\Data\2018-10-17\07-59-49-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 0\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 0.pkl


C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10
  return 10.*np.log10(x*1000.)


C:\Data\2018-10-17\09-11-22-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 1\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 1.pkl
C:\Data\2018-10-17\10-22-55-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 2\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 2.pkl
C:\Data\2018-10-17\11-34-28-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 3\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 3.pkl
C:\Data\2018-10-17\12-46-01-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 4\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 4.pkl


In [14]:
for resonator_id in range(1):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            fitter = resonator_tools.circuit.reflection_port(f_data =measurement[1][1], z_data_raw=measurement[2][-1,:])
            plt.figure()
            fitter.autofit()
            fitter.plotall()
            plt.figure()
            fitter.autofit(electric_delay=49e-9)
            fitter.plotall()
            fitter.GUIfit()
            plt.show()

C:\Data\2018-10-17\07-59-49-Res BMTSU_MISiS Nb 17.10.18 sample #0 res 0\Res BMTSU_MISiS Nb 17.10.18 sample #0 res 0.pkl


In [19]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    fitdata['single_photon_limit'] =  port1.get_single_photon_limit()
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  Jul 19 2018 20:04:21
2


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\data/2018-07-19/20-04-21-Res MISiS Nb 18.07.18 sample #1 res 0/'

# Загрузить фиты и построить графики

In [29]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 5
columns = 1
figsize=(8.27, 11.69)
figs, axes = plt.subplots(rows,columns, figsize=figsize)
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
page_id = 0
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        if resonator_id and not resonator_id%rows*columns: 
            page_id +=1
            plt.tight_layout()
            pdf.savefig(figs)
            figs, axes = plt.subplots(rows, columns, figsize=figsize)
        if rows>1 and columns>1:
            Q_ax = axes[(resonator_id%(rows*columns))%rows, (resonator_id%(rows*columns))//rows]
        else:
            Q_ax = axes[resonator_id%(rows*columns)]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = f_ax.annotate('#{7}: $f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}\n$Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])]), \
                            resonator_id), \
                          (0.2, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e3, 1e6])
        plt.grid(True)
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

    Unnamed: 0            Qc        Qc_err            Qi        Qi_err  \
0            0  1.803567e+06  2.290266e+05  1.118012e+05  2.102666e+04   
1            1  1.636693e+06  1.472408e+05  1.077181e+05  1.332000e+04   
2            2  1.578574e+06  1.293341e+05  1.086929e+05  1.393062e+04   
3            3  1.482919e+06  1.022911e+05  1.039378e+05  9.908051e+03   
4            4  1.490708e+06  1.036889e+05  1.073074e+05  1.087181e+04   
5            5  1.478953e+06  9.447183e+04  1.074052e+05  8.978962e+03   
6            6  1.478410e+06  9.175234e+04  1.088046e+05  9.728799e+03   
7            7  1.457948e+06  9.248935e+04  1.073539e+05  8.937184e+03   
8            8  1.440319e+06  8.710279e+04  1.061687e+05  8.835670e+03   
9            9  1.466809e+06  7.991139e+04  1.086396e+05  8.728418e+03   
10          10  1.437226e+06  7.685276e+04  1.054955e+05  8.435736e+03   
11          11  1.422624e+06  7.689913e+04  1.048696e+05  8.317201e+03   
12          12  1.420185e+06  7.551593

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


    Unnamed: 0            Qc        Qc_err            Qi        Qi_err  \
0            0  1.359627e+05  4.494458e+04  2.810750e+04  2.017981e+04   
1            1  1.592351e+05  7.225676e+04  2.928054e+04  1.982388e+04   
2            2  1.571380e+05  6.771302e+04  3.485286e+04  4.714475e+04   
3            3  1.804534e+05  8.924040e+04  3.755349e+04  4.820970e+04   
4            4  2.625525e+05  1.236452e+05  5.676930e+04  1.723635e+05   
5            5  3.159977e+05  4.457812e+04  6.735619e+04  1.448834e+04   
6            6  3.028821e+05  4.682036e+04  7.149148e+04  1.651980e+04   
7            7  2.847259e+05  3.853871e+04  6.682502e+04  1.526920e+04   
8            8  3.280054e+05  4.864561e+04  7.781848e+04  1.584813e+04   
9            9  2.562344e+05  2.254329e+04  6.115344e+04  8.011244e+03   
10          10  2.414265e+05  2.579833e+04  5.703796e+04  9.095126e+03   
11          11  2.578402e+05  2.167597e+04  6.253384e+04  9.222581e+03   
12          12  2.506844e+05  1.904848

In [103]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0    40169
1    35365
2    18561
3     5326
4      439
Name: Ql_lp, dtype: int32

In [104]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,fr,Ql_lp,Ql_hp,Qe,Qi_lp,Qi_hp,single_photon_limits,k
0,1.062258e+10,40169.100183,40169.100183,608533.914142,43008.045469,43008.045469,-133.536634,1.745602e+04
1,7.149959e+09,35364.866068,23292.299168,179904.213139,43319.048735,28107.504226,-138.749356,3.974315e+04
2,7.286002e+09,18560.693220,18739.620133,101733.003265,22548.612822,23863.148205,-138.620974,7.161886e+04
3,7.422990e+09,5326.245855,4521.253121,5630.345741,136394.727061,130173.279933,-138.813679,1.318390e+06
4,7.602062e+09,438.582248,2344.814944,8882.131248,3299.904538,3588.361101,-131.309599,8.558827e+05


In [88]:
figs, ax = plt.subplots(1,4, figsize=(16, 5))
c = 'fr'
ax[0].plot(summary_table_new[c], marker='o')
ax[0].set_xlabel('resonator id\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[0].set_ylabel('$f_r$')
ax[0].grid(True, which='both')
c = 'Qe'
ax[1].boxplot(summary_table_new['Qe'])
ax[1].set_yscale('log')
ax[1].grid(True, which='both')
ax[1].set_ylabel('$Q_e$')
ax[1].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[1].set_ylim(1e3, 1e6)
c = 'Qi_hp'
ax[2].boxplot(summary_table_new['Qi_hp'])
ax[2].set_ylabel('$Q_i$ (hp)')
ax[2].set_yscale('log')
ax[2].grid(True, which='both')
ax[2].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[2].set_ylim(1e3, 1e6)
c = 'Qi_lp'
ax[3].boxplot(summary_table_new['Qi_lp'])
ax[3].set_ylabel('$Q_i$ (lp)')
ax[3].set_yscale('log')
ax[3].grid(True, which='both')
ax[3].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[3].set_ylim(1e3, 1e6)
plt.tight_layout()
plt.suptitle(sample_name)
plt.savefig('{0}/{1}-boxplot.pdf'.format(save_path, sample_name))

In [143]:
print(summary_table_new['fr'].describe())

count    2.300000e+01
mean     6.471979e+09
std      6.745989e+08
min      5.389021e+09
25%      5.922199e+09
50%      6.468419e+09
75%      7.017835e+09
max      7.638156e+09
Name: fr, dtype: float64


In [229]:
rf_switch.get_ch1_switch()

0

# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')

In [28]:
%matplotlib qt5

In [378]:
names=[r"C:\Data\2018-10-12\01-04-11-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl",
  r'C:\Data\2018-10-12\01-07-54-1808_072-2 Coil #NNDAC-8 anticrossings qubit 2\1808_072-2 Coil #NNDAC-8 anticrossings qubit 2.pkl',
      r'C:\Data\2018-10-12\03-48-08-1808_072-2 Coil #NNDAC-10 anticrossings qubit 3\1808_072-2 Coil #NNDAC-10 anticrossings qubit 3.pkl',
     
       r'C:\Data\2018-10-12\03-40-43-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl'
]

In [517]:
num=2

In [518]:
f = open(names[num],'rb')
data = pickle.load(f)

In [519]:
freq=data[1]['S-parameter'][1][1]
cur = data[1]['S-parameter'][1][0]
s21 = data[1]["S-parameter"][2]

In [523]:
plt.pcolormesh(cur,freq,np.imag(s21).T)
plt.show()

In [524]:
plt.plot(freq,np.imag(s21[20]))
plt.show()

In [525]:
fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(data[1]["S-parameter"], cur, resonator_type="reflection_port")

In [526]:
fitdata

,Qc,Qc_err,Qi,Qi_err,Ql,Ql_err,chi_square,fr,fr_err,power,single_photon_limit,theta0
0,9262.989964,142.612175,102958.281835,12528.012275,8498.402452,160.928894,0.005399,6.735200e+09,4398.452779,-4.00,-142.178465,0.033198
1,10979.526530,136.986154,140700.671175,20724.511326,10184.762252,165.779408,0.005556,6.735231e+09,3933.047054,-3.92,-143.012372,0.027002
2,10555.493403,94.280144,133316.851833,13634.316960,9781.067707,112.242598,0.002682,6.735207e+09,2709.110871,-3.84,-142.832162,0.024882
3,11099.003553,183.920711,137581.722830,23891.859914,10270.463884,218.612187,0.007928,6.735204e+09,4666.945205,-3.76,-143.038188,0.024866
4,10130.893108,62.119434,120850.835909,8581.236787,9347.310573,77.615613,0.001445,6.735167e+09,2002.704481,-3.68,-142.616530,0.029770
5,10341.844877,67.772927,127413.888461,10695.147427,9565.443395,87.147707,0.001715,6.735157e+09,2054.196025,-3.60,-142.727409,0.028327
6,9891.585134,72.733032,112473.472145,8947.314550,9091.982219,94.764240,0.001933,6.735128e+09,2366.272581,-3.52,-142.479836,0.034560
7,9723.624677,88.090088,116117.435737,10183.398575,8972.289012,105.070459,0.002413,6.735126e+09,2758.064844,-3.44,-142.439110,0.034627
8,13723.385582,768.631217,185124.674081,134905.987590,12776.273942,965.008794,0.087975,6.735241e+09,12983.616509,-3.36,-144.012649,0.007494
9,10947.898680,150.763842,135517.431960,21055.132412,10129.572015,180.864744,0.006108,6.735169e+09,4013.971397,-3.28,-142.977785,0.026964


In [527]:
fr=fitdata['fr']
Q=fitdata['Qi']

In [528]:
spl=fitdata['single_photon_limit']

In [529]:
spl.mean()

-142.81925254053328

In [532]:
plt.plot(cur,Q)
plt.show()

In [531]:
plt.hist(Q)

(array([ 2.,  1.,  3.,  4., 19., 26., 30., 10.,  5.,  1.]),
 array([ 74677.85003448,  85722.53243915,  96767.21484383, 107811.89724851,
        118856.57965319, 129901.26205787, 140945.94446254, 151990.62686722,
        163035.3092719 , 174079.99167658, 185124.67408125]),
 <a list of 10 Patch objects>)

In [533]:
trh={'Qmin':[70e3,70e3,100e3],
    'Qmax':[110e3,6e5,160e3]}

In [537]:
cur_new = cur

In [432]:
trh['Qmax'][1]

150000.0

In [535]:
#indexfr= fr[(fr<6e9) | (fr>7e9)].index
#indexQ= Q[(Q<1e4) | (Q>0.75e6)].index
indexfr= fr[(fr<6.7e9) | (fr>6.8e9)].index
indexQ= Q[(Q<trh['Qmin'][2]) | (Q>trh['Qmax'][2])].index

In [538]:
indexQ

Int64Index([8, 19, 37, 62, 64, 66, 71, 74, 75, 79, 80, 82], dtype='int64')

In [304]:
indexfr

Int64Index([], dtype='int64')

In [539]:
cur_new = np.delete(cur_new,indexQ)
cur_new.shape

(89,)

In [402]:
cur_new = np.delete(cur_new,indexfr)
cur_new.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """Entry point for launching an IPython kernel.


(0,)

In [410]:
np.mean(Q)

91750.59407702721

In [543]:
np.mean(fitdata['Ql'])

9786.511281884079

In [297]:
fitdata=fitdata.drop(indexfr)

In [540]:
fitdata=fitdata.drop(indexQ)

In [312]:
plt.figure(2)
plt.plot(cur_new,fitdata["fr"])
plt.show()

In [247]:
fr.shape

(87,)

In [315]:
plt.plot(cur_new,fr)
plt.show()

In [541]:
#plt.figure()
plt.scatter(cur_new,fitdata['Qi'])
plt.xlabel("Напряжение,V")
plt.ylabel("Q_i")
plt.show()

In [509]:
np.mean(2*Q/(2*np.pi*fr))

9.00153224502442

In [329]:
np.mean(2*fitdata['Ql']/(2*np.pi*fr))

4.6240390369067573e-07